In [1]:
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier
import cPickle as pickle
np.random.seed = 0
seed = 0

In [44]:
#train = np.load('../../nn/data/train_scaled.npy')
test_sub = np.load('../../nn/data/test_scaled.npy')

In [3]:
labels = train[:,0]
train = train[:,1:]

In [24]:
test_size = 3000
#test_idx = np.random.randint(0, train.shape[0], test_size)
test_data = train[:test_size,]
test_labels = labels[:test_size,]
train_data = train[test_size:,]
train_labels = labels[test_size:,]

In [ ]:
#import sys
#sys.path.append('../py')
#import xgboost_optimizer

In [29]:
# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation

def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini



# Funcitons from olivier's kernel
# https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]


In [39]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  

params = {
    'n_estimators' : MAX_ROUNDS,
    'objective' : "binary:logistic",
    'learning_rate' : LEARNING_RATE, 
    'scale_pos_weight' : 1.6,
    'subsample' : .8,
    'colsample_bytree' : .8,
    'min_child_weight' : 6,
    'max_depth' : 4,
    'gamma' : 10,
    'reg_alpha' : 8,
    'reg_lambda' : 1.3,
    'n_jobs' : -1,
    'eval_metric' : 'auc'
}
model = XGBClassifier(**params)

In [40]:
test_pred = np.zeros(test_data.shape[0])
K = 2
kfold = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
for i, (train_indices, valid_indices) in enumerate(kfold.split(train_data,train_labels.reshape(-1))) :
    
    x_train, y_train = train_data[train_indices], train_labels[train_indices]
    x_valid, y_valid = train_data[valid_indices], train_labels[valid_indices]
    
    eval_set = [(x_valid, y_valid)]
    
    st = np.datetime64('now')
    print "\nFold ", i 
    
    _ = model.fit(x_train, y_train, eval_set=eval_set, eval_metric=gini_xgb, 
              early_stopping_rounds=EARLY_STOPPING_ROUNDS, verbose=False)
    #_ = model.fit(x_train, y_train)
    
    print( "  Best N trees = ", model.best_ntree_limit )
    print( "  Best gini = ", model.best_score )
    
    pred = model.predict_proba(x_valid)[:,1]
    print( "  Gini = ", eval_gini(y_valid, pred) )
    print "It took : ", np.datetime64('now') - st
    
    test_pred += model.predict_proba(test_data)[:,1]
test_pred /= K


Fold  0
('  Best N trees = ', 182)
('  Best gini = ', -0.276029)
('  Gini = ', 0.27599497377305682)
It took :  564 seconds

Fold  1
('  Best N trees = ', 202)
('  Best gini = ', -0.277393)
('  Gini = ', 0.27710313624184202)
It took :  597 seconds


In [ ]:
temp = test_pred

In [49]:
ids = np.load('../../nn/data/test_ids.npy')

In [50]:
with open('../data/submission.csv', 'wb') as f :
	f.write('id,target')
	for i,p in zip(ids,test_pred) :
		f.write('\n%d,%.4f'%(i,p))

In [43]:
with open('../data/model.pkl','wb') as f :
    pickle.dump(model, f)

-----------

# Draft

In [25]:
with open('../data/model.pkl', 'rb') as f:
    model = pickle.load(f)

In [47]:
test_pred = model.predict_proba(test_sub)[:,1]

In [46]:
gini_normalized(test_labels, test_pred)

0.34985846050996006

In [42]:
eval_gini(test_labels, test_pred)

0.35030414451303804